# 머신러닝 with Python 
## 09 앙상블 학습 스태킹 실습 - breast_cancer 데이터

##### 스태킹 알고리즘을 활용하여 유방암 여부를 예측하는 모형을 만들어본다.

책 p.295~ 


### 데이터 불러오기

In [1]:
from sklearn import datasets
raw_breast_cancer = datasets.load_breast_cancer()

In [2]:
# 데이터 셋 내 피처 살펴보기
raw_breast_cancer.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

### 피처, 타깃 데이터 지정

In [3]:
X = raw_breast_cancer.data
y = raw_breast_cancer.target

### 트레이닝/테스트 데이터 분할

### 데이터 표준화

In [7]:
from sklearn.model_selection import train_test_split              # 데이터 분할을 위한 함수
from sklearn.preprocessing import StandardScaler                  # 데이터 표준화를 위한 함수

X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 데이터 분할

std_scale = StandardScaler()                                      # 표준화 스케일러 선언
std_scale.fit(X_tn)                                               # 트레이닝 피처(데이터)를 기준으로 표준화 스케일러 적합

X_tn_std = std_scale.transform(X_tn)                              # 트레인, 테스트 데이터를 표준화 스케일러 처리
X_te_std = std_scale.transform(X_te)

### 데이터 학습

In [11]:
from sklearn import svm                                   # 베이스 학습기로 서포트 벡터 머신, 가우시안 나이브 베이즈
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression       # 메타 학습기로 로지스틱 회귀 분석 모형 사용
from sklearn.ensemble import StackingClassifier           # 스태킹 분류 모형


base_clf1 = svm.SVC(kernel = 'linear', random_state = 0)  # 베이스 학습기 모형 설정
base_clf2 = GaussianNB()

clf_stacking = StackingClassifier(estimators = [('svm', base_clf1),('gnb', base_clf2)],   # 개별 학습기 넣은 최종 모형
                                  final_estimator = LogisticRegression(random_state = 0)) # 분류 문제를 해결해야 할 경우 StackingRegressor 사용

clf_stacking.fit(X_tn_std, y_tn)                          # 데이터 적합

StackingClassifier(estimators=[('svm', SVC(kernel='linear', random_state=0)),
                               ('gnb', GaussianNB())],
                   final_estimator=LogisticRegression(random_state=0))

### 데이터 예측

In [12]:
pred_stacking = clf_stacking.predict(X_te_std)
print(pred_stacking)

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0]


### 정확도 평가

In [16]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_stacking)
print(accuracy)

0.965034965034965


### confusion matrix 확인

In [17]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_stacking)
print(conf_matrix)

[[50  3]
 [ 2 88]]


### 분류 리포트 확인

In [19]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_stacking)
print(class_report)

              precision    recall  f1-score   support

           0       0.96      0.94      0.95        53
           1       0.97      0.98      0.97        90

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.96      0.97      0.96       143

